In [100]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
import os
import requests  
import pyspark as ps
import re
import plotly.graph_objects as go
import plotly.express as px
from scipy import stats
from pyspark.sql.functions import udf, concat, col, lit
from numpy.core.fromnumeric import shape
from pyspark.sql.types import *
from pyspark.sql import functions as f

In [110]:
yourpath = './COVID-19/csse_covid_19_data/csse_covid_19_daily_reports_us/'

dict = {}
for root, dirs, files in os.walk(yourpath, topdown=False):
    for name in files:
        dict[name[:-4]]= pd.read_csv(yourpath+name)

In [111]:
pop_density= pd.read_csv('./Population-Density_By_State.csv', delimiter=',')  
pop_density.rename(columns={'GEO.display-label':'States'}, inplace=True)
pop_density.drop(columns=['GEO.id','GEO.id2'], axis=1, inplace=True)
pop_density.drop(index=[51],axis=0, inplace=True)

In [112]:
dc =pd.Series({'state_name':'District of Columbia','party':'democrat','name':'Muriel Bowser'})
governor= pd.read_csv('./us-governors.csv', delimiter=',')
pop_density.rename(columns={'state_name':'States'}, inplace=True)
governor.drop(columns=['state_name_slug','state_code','state_code_slug','votesmart','title','name_slug','first_name','middle_name','last_name','name_suffix','goes_by','pronunciation','gender','ethnicity','religion','openly_lgbtq','date_of_birth','entered_office','term_end','biography','phone','fax','latitude','longitude','address_complete','address_number','address_prefix','address_street','address_sec_unit_type','address_sec_unit_num','address_city','address_state','address_zipcode','address_type','website','contact_page','facebook_url','twitter_handle','twitter_url','photo_url'], axis=1, inplace=True)
poli_leader = governor.append(dc,ignore_index=True)
poli_leader = poli_leader.sort_values(by=['state_name'])

In [113]:
popu= pd.read_csv('./popstate.csv', delimiter='\t')
#popu.drop(columns=['Pop_1990','Pop_2000','Pop_2010','Change2010-19'])

In [114]:
city_pop = pd.read_csv('./PopbyCIty.csv', engine='python', delimiter='\t')

In [115]:
citypop = spark.read.options(header='True', inferSchema='True', delimiter='\t').csv('./popcity.csv')
citypop.createOrReplaceTempView('cp')
commaRep = udf(lambda x: re.sub(',','', x))
test=citypop.select('2020rank',col("estimate").alias('estimate'))
test=test.withColumn('estimate',commaRep('estimate'))
test=test.withColumn("estimate",test.estimate.cast(IntegerType()))

NameError: name 'spark' is not defined

In [116]:
c1 = spark.sql('SELECT State as States, COUNT(State) as Cities_100 FROM cp GROUP BY State ORDER BY State')
cit_100 = c1.toPandas()
ss =pd.DataFrame({'States':[' West Virginia',' Maine',' Vermont', ' Wyoming',' Delaware'], 'Cities_100':[0,0,0,0,0]})
cit_100 = cit_100.append(ss,ignore_index=True)
cit_100 = cit_100.sort_values(by=['States'])

NameError: name 'spark' is not defined

In [117]:
cit_500= spark.sql('SELECT State, COUNT(State) as Count FROM cp WHERE estimate>500000 GROUP BY State ORDER BY State')

NameError: name 'spark' is not defined

In [118]:
twentyone={}
twenty = {}

for k,v in dict.items():
    if k[-1]=='1':
        
        twentyone[k]=v
    else:
        twenty[k]=v     

In [119]:
df = pd.DataFrame()
for k,v in sorted(twenty.items()):
    df['States']= v['Province_State']
    df['Lat']= v['Lat']
    df['Long']= v['Long_']
    break
df.sort_values(by='States')
df.drop(index=[2,9,13,14,39,44,52,58], inplace=True)

In [120]:
new_df= df.merge(pop_density['Density per square mile of land area'], on=pop_density['States'], how='left')
new_df.drop(columns='key_0', inplace=True)
new_df1= new_df.merge(poli_leader['party'], on=pop_density['States'], how='left')
new_df1.drop(columns='key_0', inplace=True)
new_df2= new_df1.merge(cit_100['Cities_100'], on=pop_density['States'], how='inner')
new_df2.drop(columns='key_0', inplace=True)
new_df3= new_df2.merge(popu['Pop_2019'], on=pop_density['States'], how='inner')
new_df3.drop(columns='key_0', inplace=True)

NameError: name 'cit_100' is not defined

In [121]:
for k,v in sorted(twenty.items()):
    v.sort_values(by='Province_State')
    v.drop(index=[2,9,13,14,39,44,52], inplace=True)
    v.index=range(0,len(v))
for k,v in sorted(twentyone.items()):
    v.sort_values(by='Province_State')
    v.drop(index=[2,9,13,14,39,44,52], inplace=True)
    v.index=range(0,len(v))

In [122]:
for k,v in sorted(twenty.items()):
    #lst.append(v['Confirmed'])
    #lst.append(v['Deaths'])
    new_df3[k + ' confirmed'] = v['Confirmed']
    new_df3[k + ' deaths'] = v['Deaths']

for k,v in sorted(twentyone.items()):
    new_df3[k + ' confirmed'] = v['Confirmed']
    new_df3[k + ' deaths'] = v['Deaths']

NameError: name 'new_df3' is not defined

In [17]:
confirmed_cases=new_df3.iloc[:,7::2]
confirmed_deaths=new_df3.iloc[:,8::2]

In [69]:
confirmed_death_total=new_df3.iloc[:,-1]
confirmed_cases_total=new_df3.iloc[:,-2]
new_df3['cd_percentage'] = confirmed_death_total/new_df3['Pop_2019']
new_df3['cc_percentage'] = confirmed_cases_total/new_df3['Pop_2019']
new_df3

,States,Lat,Long,Density per square mile of land area,party,Cities_100,Pop_2019,04-13-2020 confirmed,04-13-2020 deaths,04-14-2020 confirmed,...,06-03-2021 confirmed,06-03-2021 deaths,06-04-2021 confirmed,06-04-2021 deaths,06-05-2021 confirmed,06-05-2021 deaths,06-06-2021 confirmed,06-06-2021 deaths,cd_percentage,cc_percentage
50,Wyoming,42.7560,-107.3025,5.8,republican,0,578759,275,1,282,...,60549,720,60628,720,60628,720,60628,720,1.281459e-76,2.214149e-82
48,West Virginia,38.4912,-80.9545,77.1,democrat,0,1792147,611,8,640,...,162111,2807,162232,2813,162232,2813,162232,2813,4.601567e-83,2.567628e-89
45,Vermont,44.0459,-72.7107,67.9,republican,0,623989,748,28,752,...,24240,255,24252,256,24252,256,24252,256,1.787601e-77,2.864795e-83
19,Maine,44.6939,-69.3819,43.1,democrat,0,1344212,698,19,735,...,67986,837,68058,838,68155,839,68233,839,1.085040e-81,8.071940e-88
7,Delaware,39.3185,-75.5071,460.8,democrat,0,973764,1758,54,1926,...,108957,1668,109005,1674,109070,1677,109126,1677,1.583347e-79,1.626007e-85
26,Montana,46.9219,-110.4544,6.8,republican,1,1068778,394,7,399,...,112129,1627,112260,1632,112260,1632,112260,1632,4.423846e-80,4.139162e-86
24,Mississippi,32.7416,-89.6787,63.2,republican,1,2976149,2942,98,3087,...,318048,7324,318189,7325,318189,7325,318189,7325,7.439089e-86,2.499569e-92
20,Maryland,39.0639,-76.8021,594.8,republican,1,6045680,8936,262,9472,...,460194,9626,460339,9630,460471,9639,460575,9645,5.285282e-90,8.742246e-97
34,North Dakota,47.5289,-99.7840,9.7,republican,1,762062,331,8,341,...,110098,1546,110151,1546,110178,1547,110199,1547,4.946675e-78,6.491171e-84
11,Hawaii,21.0943,-157.4983,211.8,democrat,1,1415872,504,9,511,...,36402,501,36471,501,36535,502,36616,505,2.814071e-82,1.987518e-88


In [109]:
rep_idx = new_df3[new_df3['party']=='republican'].index
dem_idx = new_df3[new_df3['party']!='republican'].index

NameError: name 'new_df3' is not defined

In [108]:
dens_idx = new_df3[new_df3['Density per square mile of land area']<=200].index
dens200_idx = new_df3[new_df3['Density per square mile of land area']>=200].index

NameError: name 'new_df3' is not defined

In [107]:
rep = confirmed_cases.T.pct_change().mean()[rep_idx]
dem = confirmed_cases.T.pct_change().mean()[dem_idx]
stats.ttest_ind(rep,dem)

NameError: name 'rep_idx' is not defined

In [99]:
rep = confirmed_deaths.T.pct_change().mean()[rep_idx]
dem = confirmed_deaths.T.pct_change().mean()[dem_idx]
print(rep)
stats.ttest_ind(rep,dem)

NameError: name 'rep_idx' is not defined

In [74]:
below = confirmed_deaths.T.pct_change().mean()[dens1_idx ]
over = confirmed_deaths.T.pct_change().mean()[dens100_idx]
print(len(below))
print(len(over))
stats.ttest_ind(below,over)

25
26


Ttest_indResult(statistic=4.292505369372853, pvalue=8.309748103503926e-05)

In [14]:
final_df = pd.read_csv('/home/aa1/Covid-cause-and-effect/Final_df.csv')
confirmed_cases=final_df.iloc[:,8::2]
confirmed_deaths=final_df.iloc[:,9::2]

In [69]:
ct = confirmed_cases.T
df1 = pd.Series(final_df['States'])
ct.columns=df1
ct

States,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,Florida,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
04-13-2020 confirmed,3870.000000,277.000000,3705.000000,1410.000000,2.395600e+04,7691.000000,13381.000000,1758.000000,1955.00000,2.101900e+04,...,868.000000,5610.000000,1.432800e+04,2363.000000,748.00000,5747.000000,10635.000000,611.00000,3428.000000,275.000000
04-14-2020 confirmed,4041.000000,285.000000,3809.000000,1498.000000,2.542700e+04,7950.000000,13989.000000,1926.000000,2058.00000,2.162800e+04,...,988.000000,5827.000000,1.508800e+04,2417.000000,752.00000,6182.000000,10799.000000,640.00000,3555.000000,282.000000
04-15-2020 confirmed,4307.000000,293.000000,3964.000000,1569.000000,2.671900e+04,7956.000000,14755.000000,2014.000000,2197.00000,2.251100e+04,...,1168.000000,5827.000000,1.604400e+04,2548.000000,759.00000,6500.000000,10942.000000,702.00000,3721.000000,287.000000
04-16-2020 confirmed,4465.000000,300.000000,4237.000000,1620.000000,2.768700e+04,8286.000000,15884.000000,2070.000000,2350.00000,2.334300e+04,...,1311.000000,6375.000000,1.698500e+04,2683.000000,774.00000,6889.000000,11057.000000,728.00000,3875.000000,296.000000
04-17-2020 confirmed,4557.000000,309.000000,4511.000000,1695.000000,2.947000e+04,8691.000000,16809.000000,2317.000000,2476.00000,2.475900e+04,...,1411.000000,6263.000000,1.792500e+04,2793.000000,779.00000,7491.000000,11517.000000,775.00000,4053.000000,305.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
06-03-2021 confirmed,545028.000000,70408.000000,882691.000000,341889.000000,3.792962e+06,545002.000000,347748.000000,108957.000000,49041.00000,2.329867e+06,...,124242.000000,863620.000000,2.959171e+06,406825.000000,24240.00000,676041.000000,438544.000000,162111.00000,674939.000000,60549.000000
06-04-2021 confirmed,545585.000000,70469.000000,883037.000000,342142.000000,3.794129e+06,545769.000000,347891.000000,109005.000000,49068.00000,2.329867e+06,...,124261.000000,863840.000000,2.961257e+06,407113.000000,24252.00000,676300.000000,439263.000000,162232.00000,675150.000000,60628.000000
06-05-2021 confirmed,545585.000000,70469.000000,883314.000000,342345.000000,3.794927e+06,546500.000000,347891.000000,109070.000000,49068.00000,2.332867e+06,...,124261.000000,863840.000000,2.962395e+06,407419.000000,24252.00000,676586.000000,440057.000000,162232.00000,675150.000000,60628.000000
06-06-2021 confirmed,545585.000000,70469.000000,883821.000000,342433.000000,3.795580e+06,546874.000000,347891.000000,109126.000000,49068.00000,2.332867e+06,...,124261.000000,863840.000000,2.962934e+06,407642.000000,24252.00000,676741.000000,440057.000000,162232.00000,675150.000000,60628.000000


In [70]:
ct.drop(index='cd_percentage',axis=0, inplace=True)

In [125]:
fig = px.line(ct, title="Covid Cases Over Time by State")
fig.update_traces(mode="markers+lines", hovertemplate=None)
fig.update_layout(hovermode="x")
fig.update_layout(legend_orientation='h')
fig.update_yaxes(type="log")
fig.update_layout(
    autosize=False,
    width=1500,
    height=850,
    paper_bgcolor="LightSteelBlue")
fig.write_html("/home/aa1/Covid-cause-and-effect/SuperGraph5000.html")